In [39]:
# Imports

import pandas as pd
import cpi

In [40]:
# Loading datasets from FRED
income = pd.read_csv('./datasets/medianHHIncome.csv')
housePrices = pd.read_csv('./datasets/medianHomePrice.csv')

In [41]:
# Adjusting date formats for income dataset
income['date'] = pd.to_datetime(income['observation_date']).dt.year
income = income.rename(columns={'MEHOINUSA646N' : "medianHHIncome"})
income.head()

,observation_date,medianHHIncome,date
0,1984-01-01,22420,1984
1,1985-01-01,23620,1985
2,1986-01-01,24900,1986
3,1987-01-01,26060,1987
4,1988-01-01,27230,1988


In [42]:
# Adjusting date and income formats to yearly for median HH income dataset
length = int(len(housePrices)/4)

dates = pd.date_range("19630101", periods=length, freq='Y')

observation_date = [0] * length

for i in range(0, length):
    observation_date[i] = dates[i].year
    
newPrices = pd.DataFrame({'date': observation_date})

# Averaging quarterly incomes
avgYearPrice = [0] * length
for i in range(0, length):
    start = i*4
    newPrice = housePrices['medianPrice'][start:start+4].mean()
    avgYearPrice[i] = newPrice

newPrices['avgYearPrice'] = avgYearPrice
newPrices = newPrices[newPrices['date'] >= income['date'][0]]
newPrices = newPrices[newPrices['date'] <= income['date'][len(income)-1]]

In [43]:
# Joining datasets
df = pd.merge(income, newPrices, on="date")
df.head()

,observation_date,medianHHIncome,date,avgYearPrice
0,1984-01-01,22420,1984,79950.0
1,1985-01-01,23620,1985,84275.0
2,1986-01-01,24900,1986,92025.0
3,1987-01-01,26060,1987,104700.0
4,1988-01-01,27230,1988,112225.0


In [44]:
# Inflation Adjustments

df["realMedianHHIncome"] = df.apply(
    lambda row: cpi.inflate(row["medianHHIncome"], row["date"], to=2023),
    axis=1
)

df["realHousePrice"] = df.apply(
    lambda row: cpi.inflate(row["avgYearPrice"], row["date"], to=2023),
    axis=1
)

In [45]:
# Checking Data
df.head()

,observation_date,medianHHIncome,date,avgYearPrice,realMedianHHIncome,realHousePrice
0,1984-01-01,22420,1984,79950.0,65749.940712,234465.109721
1,1985-01-01,23620,1985,84275.0,66887.186245,238650.195632
2,1986-01-01,24900,1986,92025.0,69225.180657,255841.255018
3,1987-01-01,26060,1987,104700.0,69899.067958,280830.100352
4,1988-01-01,27230,1988,112225.0,70135.549112,289054.792477


In [46]:
# Saving to CSV
df.to_csv('incomeAndPrice.csv', index=False)